In [1]:
import SimpleITK as sitk
import numpy as np

In [7]:
file = "../data/subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260.mhd"
ct_mhd = sitk.ReadImage(file)
ct_arr = np.array(sitk.GetArrayFromImage(ct_mhd), np.float32)
ct = np.clip(ct_arr, -1000, 1000)

In [8]:
ct_shape = ct.shape
slice_list = []
slice_dimensions = [48, 32, 32]
center_irc = [50, 40, 40]
for i, dim in enumerate(slice_dimensions):
    start_idx = center_irc[i] + dim//2
    end_idx = start_idx + dim

    if start_idx < 0:
        start_idx = 0
        end_idx = dim

    if end_idx > ct_shape[i]:
        start_idx = ct_shape[i] - dim
        end_idx = ct_shape[i]

    slice_list.append(slice(start_idx, end_idx))

ct_slice = ct[tuple(slice_list)]

In [10]:
ct_slice.shape

(48, 32, 32)

In [12]:
import torch
import torch.nn.functional as F

In [32]:
dummy = torch.rand(4, 3, 3).unsqueeze(0).unsqueeze(0).to(torch.float32)
print(dummy.shape)
dummy

torch.Size([1, 1, 4, 3, 3])


tensor([[[[[0.8454, 0.8988, 0.2442],
           [0.0788, 0.5924, 0.8540],
           [0.0335, 0.1380, 0.4017]],

          [[0.3564, 0.2630, 0.3144],
           [0.7082, 0.6668, 0.1703],
           [0.3695, 0.5085, 0.4956]],

          [[0.9106, 0.4787, 0.7329],
           [0.1373, 0.2357, 0.7227],
           [0.7239, 0.6273, 0.9940]],

          [[0.9333, 0.9122, 0.5482],
           [0.6726, 0.6906, 0.9040],
           [0.0336, 0.8657, 0.5447]]]]])

In [33]:
transform_t = torch.eye(4)
transform_t

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [34]:
# flip
for i in range(3):
    transform_t[i, i] *= -1

flipped_t = transform_t
flipped_t

tensor([[-1.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.],
        [ 0.,  0., -1.,  0.],
        [ 0.,  0.,  0.,  1.]])

In [35]:
transform_t[:3].unsqueeze(0)

tensor([[[-1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.],
         [ 0.,  0., -1.,  0.]]])

In [38]:
affine_t = F.affine_grid(
            transform_t[:3].unsqueeze(0).to(torch.float32),
            dummy.size(),
            align_corners=False,
        )
affine_t.shape

torch.Size([1, 4, 3, 3, 3])

In [39]:
dummy_flipped = F.grid_sample(
            dummy,
            affine_t,
            padding_mode='border',
            align_corners=False,
        ).to('cpu')

In [40]:
dummy_flipped

tensor([[[[[0.5447, 0.8657, 0.0336],
           [0.9040, 0.6906, 0.6726],
           [0.5482, 0.9122, 0.9333]],

          [[0.9940, 0.6273, 0.7239],
           [0.7227, 0.2357, 0.1373],
           [0.7329, 0.4787, 0.9106]],

          [[0.4956, 0.5085, 0.3695],
           [0.1703, 0.6668, 0.7082],
           [0.3144, 0.2630, 0.3564]],

          [[0.4017, 0.1380, 0.0335],
           [0.8540, 0.5924, 0.0788],
           [0.2442, 0.8988, 0.8454]]]]])

In [41]:
dummy

tensor([[[[[0.8454, 0.8988, 0.2442],
           [0.0788, 0.5924, 0.8540],
           [0.0335, 0.1380, 0.4017]],

          [[0.3564, 0.2630, 0.3144],
           [0.7082, 0.6668, 0.1703],
           [0.3695, 0.5085, 0.4956]],

          [[0.9106, 0.4787, 0.7329],
           [0.1373, 0.2357, 0.7227],
           [0.7239, 0.6273, 0.9940]],

          [[0.9333, 0.9122, 0.5482],
           [0.6726, 0.6906, 0.9040],
           [0.0336, 0.8657, 0.5447]]]]])